In [2]:
import xarray as xr 
import pandas as pd
import numpy as np
import os

## Functions

In [3]:
## Function to read and save forecast data ##
## Create function to read data and save it in RAM as xarray Dataset
def read_trim_forecast(file_path, va):
    """
    Args:
        file_path (str): directory of 
        va (str): variable we will be looking for 

    Return:
        forecast_data (xarray.core.dataset.Dataset): trimmed dataset
    """
    try:
        forecast_data = xr.open_dataset(file_path)[va]
        
        # if initialization_date != None:
           # forecast_data = forecast_data.sel(time=initialization_date)
        # else:
            #forecast_data = forecast_data.sel(time=-1)

        return forecast_data
    
    except KeyError:
        print(f'Variable {va} not found in dataset')

In [4]:
## Function to read and save hindcast data ##
## Create function to read data and save it as netcdf file ##
def read_trim_hindcast(file_path, va):
    """
    Args:
        file_path (str): file path to hindcast data
        va (str): variable we will be looking for in the datatset
    Returns:
        hindcast_data (xarray.core.dataset.Dataset):
    """

    try:
        hindcast_data = xr.open_mfdataset(file_path)[va].chunk(dict(time=-1))
        return hindcast_data
    except KeyError:
        print(f'Variable {va} not found in dataset')

In [5]:
def initialize_climatology(hindcast_data_file_path, variable):
    """
    Initialize climatology for the given variable from hindcast data.
    """
    climatology = xr.open_mfdataset(hindcast_data_file_path)[variable]
    climatology = climatology.groupby('time.month').mean(dim='time')
    return climatology

In [13]:
def get_anomaly(forecast_data_file_path, climatology, variable):
    """
    Calculate the anomaly for the given variable using forecast data and climatology.
    """
    forecast_data = xr.open_dataset(forecast_data_file_path)[variable]
    anomaly = forecast_data - climatology
    return anomaly

## Executable

In [6]:
workspace_directory = os.getcwd()
surface_model_file_path = rf"C:\Users\Kris\Documents\amazonforecast\data\hindcast\amazon_forecast"
forecast_data_file_path = surface_model_file_path + rf"/ldas_fcst_2024_dec01.nc"

year = 2024
month = 'dec'

hindcast_data_file_path = []
for i in range(2001, year):
    hindcast_data_file_path.append(surface_model_file_path + os.path.join(rf"/ldas_fcst_{i}_{month}01.nc"))

hindcast_data_file_path

['C:\\Users\\Kris\\Documents\\amazonforecast\\data\\hindcast\\amazon_forecast/ldas_fcst_2001_dec01.nc',
 'C:\\Users\\Kris\\Documents\\amazonforecast\\data\\hindcast\\amazon_forecast/ldas_fcst_2002_dec01.nc',
 'C:\\Users\\Kris\\Documents\\amazonforecast\\data\\hindcast\\amazon_forecast/ldas_fcst_2003_dec01.nc',
 'C:\\Users\\Kris\\Documents\\amazonforecast\\data\\hindcast\\amazon_forecast/ldas_fcst_2004_dec01.nc',
 'C:\\Users\\Kris\\Documents\\amazonforecast\\data\\hindcast\\amazon_forecast/ldas_fcst_2005_dec01.nc',
 'C:\\Users\\Kris\\Documents\\amazonforecast\\data\\hindcast\\amazon_forecast/ldas_fcst_2006_dec01.nc',
 'C:\\Users\\Kris\\Documents\\amazonforecast\\data\\hindcast\\amazon_forecast/ldas_fcst_2007_dec01.nc',
 'C:\\Users\\Kris\\Documents\\amazonforecast\\data\\hindcast\\amazon_forecast/ldas_fcst_2008_dec01.nc',
 'C:\\Users\\Kris\\Documents\\amazonforecast\\data\\hindcast\\amazon_forecast/ldas_fcst_2009_dec01.nc',
 'C:\\Users\\Kris\\Documents\\amazonforecast\\data\\hindcast\\am

In [9]:
## Global variables
surface_variable_short = ['Rainf_tavg', 'Qair_f_tavg',
                          'Qs_tavg','Evap_tavg', 'Tair_f_tavg',
                          'SoilMoist_inst', 'SoilTemp_inst', 'Streamflow_tavg']

surface_variable_long = ['precipitation rate', 'specific humidity', 'surface runoff',
                         'total evapotranspiration', 'avg. air temperature', 'soil moisture content', 'soiltemp inst', 'stream flow']
"""
surface_variable_unit = {'Rainf_tavg':'mm/day',
                         'Qair_f_tavg':'g/kg',
                         'Qs_tavg':'mm/day', 
                         'Evap_tavg':'mm/day',
                         'Tair_f_tavg':'degree Celsius', 
                         'SoilMoist_inst':'m^3 m-3', 
                         'SoilTemp_inst':'degree Celsius'}
"""

"\nsurface_variable_unit = {'Rainf_tavg':'mm/day',\n                         'Qair_f_tavg':'g/kg',\n                         'Qs_tavg':'mm/day', \n                         'Evap_tavg':'mm/day',\n                         'Tair_f_tavg':'degree Celsius', \n                         'SoilMoist_inst':'m^3 m-3', \n                         'SoilTemp_inst':'degree Celsius'}\n"

In [10]:
for variable, variable_longname in zip(surface_variable_short, surface_variable_long):
    
    print(variable_longname)

    hindcast = read_trim_hindcast(hindcast_data_file_path, va=variable)

    initialization_date = '2024-12-31'
    forecast  = read_trim_forecast(forecast_data_file_path, variable)

    climatology = initialize_climatology(hindcast_data_file_path, variable)

    #anomaly = get_anomaly(forecast_data_file_path, climatology, variable)

    file_savepath = workspace_directory + '/get_deterministic_climatology/deterministic_' + initialization_date.replace('-','_')
    #create_directory(file_savepath)
    #anomaly.to_netcdf(file_savepath + '_deterministic_anomaly_' + str(variable) + '.nc')

    climatology.to_netcdf(file_savepath + '_climatology_' + str(variable) + '.nc')
    print('Saved climatology values for ' + str(variable) + '!')

precipitation rate
Saved climatology values for Rainf_tavg!
specific humidity
Saved climatology values for Qair_f_tavg!
surface runoff
Saved climatology values for Qs_tavg!
total evapotranspiration
Saved climatology values for Evap_tavg!
avg. air temperature
Saved climatology values for Tair_f_tavg!
soil moisture content
Saved climatology values for SoilMoist_inst!
soiltemp inst
Saved climatology values for SoilTemp_inst!
stream flow
Saved climatology values for Streamflow_tavg!
